In [1]:
%pip install tensorflow

In [6]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install sklearn


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [2]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        print("Found a GPU with the name:", gpu)
else:
    print("Failed to detect a GPU.")


In [2]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add,AveragePooling2D,Dense, AvgPool2D,BatchNormalization, ReLU, DepthwiseConv2D, Reshape, Permute,Conv2D, MaxPool2D, GlobalAveragePooling2D, concatenate

In [3]:
import os
import shutil
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [4]:
def channel_shuffle(x, groups):  
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups

    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    return x

In [5]:
def shuffle_unit(x, groups, channels,strides):

    y = x
    x = Conv2D(channels//4, kernel_size = 1, strides = (1,1),padding = 'same', groups=groups)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = channel_shuffle(x, groups)
    
    x = DepthwiseConv2D(kernel_size = (3,3), strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)

    if strides == (2,2):
        channels = channels - y.shape[-1]
    x = Conv2D(channels, kernel_size = 1, strides = (1,1),padding = 'same', groups=groups)(x)
    x = BatchNormalization()(x)

    if strides ==(1,1):
        x =Add()([x,y])
    if strides == (2,2):
        y = AvgPool2D((3,3), strides = (2,2), padding = 'same')(y)
        x = concatenate([x,y])
    
    x = ReLU()(x)


    return x

In [6]:
def Shuffle_Net(nclasses, start_channels ,input_shape = (224,224,3)):

    groups = 2
    input = Input (input_shape)

    x =  Conv2D (24,kernel_size=3,strides = (2,2), padding = 'same', use_bias = True)(input)
    x =  BatchNormalization()(x)
    x =  ReLU()(x)
    
    x = MaxPool2D (pool_size=(3,3), strides = 2, padding='same')(x)

    repetitions = [3,7,3]

    for i,repetition in enumerate(repetitions):
        channels = start_channels * (2**i)

        x  = shuffle_unit(x, groups, channels,strides = (2,2))

        for i in range(repetition):
            x = shuffle_unit(x, groups, channels,strides=(1,1))

    x = GlobalAveragePooling2D()(x)

    output = Dense(n_classes,activation='softmax')(x)

    model = Model(input, output)
    return model

In [7]:
n_classes = 6
input_shape = (224,224,3)
start_channels = 32

model = Shuffle_Net(n_classes, start_channels,input_shape)
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 24)         672       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 24)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 112, 112, 24)         0         ['batch_normalization[0]

In [7]:
import pickle

# Load features_reshaped from the pickle file
with open('split_data1.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [8]:

# Set the paths to your dataset folders
qr_code_path1 = r'C:\Users\ilfas\OneDrive\Documents\college\design project\implementation\qr_codes2'
target_path = r'C:\Users\ilfas\OneDrive\Documents\college\design project\implementation\output_qr_codes'

# Function to load images without preprocessing
def load_images(directory):
    images = []
    for img in os.listdir(directory):
        img_path = os.path.join(directory, img)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Adjust the size according to your needs
        img = img[..., ::-1]  # BGR to RGB (since MobileNetV2 expects RGB)
        img = img / 255.0  # Normalize pixel values to the range [0, 1]
        images.append(img)
    return np.array(images)

# Load QR code images and target images
qr_code_images = load_images(qr_code_path1)
target_images = load_images(target_path)

# Assuming you have labels for your QR code images (modify accordingly)
labels = np.array([1] * 1000 + [2] * 1000 + [3] * 1000 + [4] * 1000 + [5] * 1000 + [6] * 1000)

# Subtract 1 to bring labels into the range [0, 5]
labels = labels - 1






In [9]:
# Split the dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(qr_code_images, labels, test_size=0.3, random_state=0)

In [10]:
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

# Define optimizer, loss function, and metrics
#rmsprop
optimizer = Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999)  # You can adjust the learning rate as needed
loss_function = SparseCategoricalCrossentropy()
metrics = [SparseCategoricalAccuracy()]

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 24)         672       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 24)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 112, 112, 24)         0         ['batch_normalization[0][0

In [14]:
from tensorflow.keras import callbacks, optimizers
import time

# Define a custom callback to measure training time
class TimingCallback(callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        end_time = time.time()
        training_time = end_time - self.start_time
        print(f"Training time: {training_time} seconds")

# Define a custom callback to measure training time
timing_callback = TimingCallback()

# Assuming X_train, y_train, X_val, y_val are your training and validation data
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=16, batch_size=32, callbacks=[timing_callback])


Epoch 1/16
132/132 [==============================] - 116s 876ms/step - loss: 0.5313 - sparse_categorical_accuracy: 0.7862 - val_loss: 3.4078 - val_sparse_categorical_accuracy: 0.3683
Epoch 2/16
132/132 [==============================] - 116s 875ms/step - loss: 0.5287 - sparse_categorical_accuracy: 0.7836 - val_loss: 1.6286 - val_sparse_categorical_accuracy: 0.5400
Epoch 3/16
132/132 [==============================] - 1010s 8s/step - loss: 0.5249 - sparse_categorical_accuracy: 0.7874 - val_loss: 2.3309 - val_sparse_categorical_accuracy: 0.4039
Epoch 4/16
132/132 [==============================] - 118s 898ms/step - loss: 0.4849 - sparse_categorical_accuracy: 0.8133 - val_loss: 5.9751 - val_sparse_categorical_accuracy: 0.2322
Epoch 5/16
132/132 [==============================] - 119s 900ms/step - loss: 0.4679 - sparse_categorical_accuracy: 0.8157 - val_loss: 2.4909 - val_sparse_categorical_accuracy: 0.4172
Epoch 6/16
132/132 [==============================] - 120s 910ms/step - loss: 0.47

In [15]:
# Save the model
model.save('C:/Users/ilfas/OneDrive/Documents/college/design project/implementation/ShuffleNet.h5')

In [10]:
# Alternatively, you can loop through the layers and print their names and shapes
for layer in model.layers:
    print(layer.name, layer.output_shape)

input_1 [(None, 224, 224, 3)]
conv2d (None, 112, 112, 24)
batch_normalization (None, 112, 112, 24)
re_lu (None, 112, 112, 24)
max_pooling2d (None, 56, 56, 24)
conv2d_1 (None, 56, 56, 8)
batch_normalization_1 (None, 56, 56, 8)
re_lu_1 (None, 56, 56, 8)
reshape (None, 56, 56, 4, 2)
permute (None, 56, 56, 2, 4)
reshape_1 (None, 56, 56, 8)
depthwise_conv2d (None, 28, 28, 8)
batch_normalization_2 (None, 28, 28, 8)
conv2d_2 (None, 28, 28, 8)
batch_normalization_3 (None, 28, 28, 8)
average_pooling2d (None, 28, 28, 24)
concatenate (None, 28, 28, 32)
re_lu_2 (None, 28, 28, 32)
conv2d_3 (None, 28, 28, 8)
batch_normalization_4 (None, 28, 28, 8)
re_lu_3 (None, 28, 28, 8)
reshape_2 (None, 28, 28, 4, 2)
permute_1 (None, 28, 28, 2, 4)
reshape_3 (None, 28, 28, 8)
depthwise_conv2d_1 (None, 28, 28, 8)
batch_normalization_5 (None, 28, 28, 8)
conv2d_4 (None, 28, 28, 32)
batch_normalization_6 (None, 28, 28, 32)
add (None, 28, 28, 32)
re_lu_4 (None, 28, 28, 32)
conv2d_5 (None, 28, 28, 8)
batch_normalization

In [16]:
# Directory path containing QR code images
qr_code_path = 'qr_codes2'

# Function to load images
def load_images(directory):
    images = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize images to match MobileNetV2 input shape
        images.append(img)
    return np.array(images)

# Load QR code images
qr_code_images = load_images(qr_code_path)
labels = np.array([1] * 1000 + [2] * 1000 + [3] * 1000 + [4] * 1000 + [5] * 1000 + [6] * 1000)

# Subtract 1 to bring labels into the range [0, 5]
labels = labels - 1

# Train the ShuffleNet model on QR code images
model.fit(qr_code_images, labels, epochs=16, batch_size=32, validation_split=0.2)


Epoch 1/16
150/150 [==============================] - 86s 463ms/step - loss: 0.6514 - sparse_categorical_accuracy: 0.7642 - val_loss: 31.8339 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/16
150/150 [==============================] - 68s 451ms/step - loss: 0.4824 - sparse_categorical_accuracy: 0.8188 - val_loss: 12.4875 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/16
150/150 [==============================] - 67s 444ms/step - loss: 0.4326 - sparse_categorical_accuracy: 0.8375 - val_loss: 5.4109 - val_sparse_categorical_accuracy: 0.1550
Epoch 4/16
150/150 [==============================] - 65s 431ms/step - loss: 0.4112 - sparse_categorical_accuracy: 0.8456 - val_loss: 7.1660 - val_sparse_categorical_accuracy: 0.1517
Epoch 5/16
150/150 [==============================] - 65s 432ms/step - loss: 0.3977 - sparse_categorical_accuracy: 0.8498 - val_loss: 6.9073 - val_sparse_categorical_accuracy: 0.1233
Epoch 6/16
150/150 [==============================] - 63s 422ms/step - loss

In [25]:
# Save the model
model.save('C:/Users/ilfas/OneDrive/Documents/college/design project/implementation/ShuffleNetwhole.h5')

c:\Users\ilfas\anaconda3\envs\py310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = Model(inputs=model.input, outputs=model.layers[-3].output)  # Get the second-to-last layer (softmax layer)
features = model.predict(qr_code_images)

# Reshape the features to have only two dimensions
num_samples, height, width, num_channels = features.shape
features_reshaped = features.reshape(num_samples, height * width * num_channels)

# Split the extracted features into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_reshaped, labels, test_size=0.3, random_state=42)

In [18]:
model = Model(inputs=model.input, outputs=model.layers[-2].output)  # Get the second-to-last layer (softmax layer)
features = model.predict(qr_code_images)

# Reshape the features to have only two dimensions
num_samples, height, width, num_channels = features.shape
features_reshaped = features.reshape(num_samples, height * width * num_channels)


# Split the extracted features into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_reshaped, labels, test_size=0.3, random_state=42)

188/188 [==============================] - 21s 107ms/step


In [37]:
import pickle

# Save features_reshaped to a file using pickle
with open('features_reshaped.pkl', 'wb') as f:
    pickle.dump(features_reshaped, f)


In [18]:
# Print the number of features
num_features = features_reshaped.shape[1]
print("Number of features:", num_features)


Number of features: 6272


In [19]:
#softmax new
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# Train SVM classifier using the extracted features
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.6233333333333333

In [12]:
#softmax
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# Train SVM classifier using the extracted features
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.5883333333333334

In [20]:
#convu new
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# Train SVM classifier using the extracted features
svm_classifier = SVC(kernel='linear',class_weight='balanced')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.6216666666666667

In [32]:
#convu
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# Train SVM classifier using the extracted features
svm_classifier = SVC(kernel='linear',class_weight='balanced')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.5727777777777778

In [16]:
svm_accuracy

0.5544444444444444

In [30]:
svm_classifier = SVC(kernel='poly', degree=2, C=1.0, gamma='auto')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.6255555555555555

In [21]:
#new
svm_classifier = SVC(kernel='poly', degree=2, C=1.0, gamma='auto')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.6627777777777778

In [31]:
# Train kNN classifier using the extracted features
knn_classifier = KNeighborsClassifier(metric='euclidean')
knn_classifier.fit(X_train, y_train)

knn_accuracy = knn_classifier.score(X_test, y_test)
knn_accuracy

0.615

In [22]:
#new
# Train kNN classifier using the extracted features
knn_classifier = KNeighborsClassifier(metric='euclidean')
knn_classifier.fit(X_train, y_train)

knn_accuracy = knn_classifier.score(X_test, y_test)

In [23]:
knn_accuracy

0.6022222222222222

In [16]:
# Train kNN classifier using the extracted features
knn_classifier = KNeighborsClassifier(metric='manhattan')
knn_classifier.fit(X_train, y_train)

knn_accuracy = knn_classifier.score(X_test, y_test)
knn_accuracy

0.6227777777777778

In [24]:
#new
# Train kNN classifier using the extracted features
knn_classifier = KNeighborsClassifier(metric='minkowski')
knn_classifier.fit(X_train, y_train)

knn_accuracy = knn_classifier.score(X_test, y_test)
knn_accuracy

0.6022222222222222

In [19]:
# Define the objective function for the HHO algorithm
def objective_function(selected_features):
    # Threshold the selected_features to be binary (0 or 1)
    selected_features = np.where(selected_features > 0.5, 1, 0)
    
    # Train SVM classifier using selected features
    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(X_train[:, selected_features], y_train)
    
    # Evaluate SVM classifier on the test set
    svm_accuracy = svm_classifier.score(X_test[:, selected_features], y_test)
    
    return -svm_accuracy  # Return negative accuracy for minimization (HHO maximizes the objective function)


# Harris Hawks Optimization Algorithm
import HHO

selected_features = HHO.HHO(objective_function, lb=-100,ub=100,SearchAgents_no=50, Max_iter=100, dim=X_train.shape[1])

HHO is now tackling  "objective_function"


KeyboardInterrupt: 